# Notebook for preparing a propagation in a Gaussian jet

This notebook shows a way to generate an input file for the multiscale model consisting of:
* The pulse propagation
* TDSE
* Hankel transform

Here we show the standard operation with a pre-defined gas.$^\dagger$ The density profile is customised in this notebook.

$^\dagger$ Therefore, all the material constants are sourced from default values within the codes. 

## Load libraries

In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import os
import h5py
import sys
import copy
import MMA_administration as MMA
import mynumerics as mn
import units
import HHG
from IPython.display import display, Markdown


%matplotlib inline
# import mpld3
# mpld3.enable_notebook()

The file where the input parameters are written:

In [ ]:
h5file = os.path.join('/mnt','d','git','MMA-interactive','results.h5')

## Physical parameters

The philosophy of our input uses a "reference Gaussian beam". This means that our reference is the Gaussian beam with known parameters (focus, waist, focus intensity) propagating in vacuum. Then we add a medium in the path of the beam in our experiment. Finally, we specify the parameters of ht XUV camera.

### Laser parameters

In [ ]:
reference_Gaussian_focus = 1.0e-2 # [m]
reference_Gaussian_waist = 110e-6 # [m]
reference_Gaussian_focus_intensity = 3.6e18 # [W/m2]
laser_wavelength = 800e-9 # [m]
laser_pulse_duration = 30e-15 # [s] (defined via 1/e in the electric field amplitude)

### Medium parameters

In [ ]:
# gas specifiers
gas = 'Kr'
medium_length = 15e-3 # [m]
ionisation_model = 'PPT'
XUV_dispersion_tables = 'NIST'


medium_pressure = 25e-3 # bar



In [ ]:
number_of_points_in_r      = 1024
number_of_points_in_t      = 1024

operators_t                =	2
first_delta_z     =	0.01 # [mm]
phase_threshold_for_decreasing_delta_z = 0.002	# [rad]

length_of_window_for_r_normalized_to_beamwaist = 4.   # [-]
length_of_window_for_t_normalized_to_pulse_duration = 6. # [-]

number_of_absorber_points_in_time = 16  # [-]

physical_output_distance_for_plasma_and_Efield = 0.00001   # [m]

output_distance_in_z_steps_for_fluence_and_power   = 100  # [-]

radius_for_diagnostics = 0.1 # [cm]

run_time_in_hours = 5.0 # [h] 

In [ ]:
reference_Gaussian_focus = medium_length/2. # [m]
reference_Gaussian_waist = 100e-6 # [m]
reference_Gaussian_focus_intensity = 1.3e18 # [W/m2]
laser_wavelength = 800e-9 # [m]
laser_pulse_duration = 45e-15 # [s] (defined via 1/e in the electric field amplitude)

In [ ]:
## Code to generate the following text ##
zR = (np.pi*reference_Gaussian_waist**2)/laser_wavelength
dr_CUPRAD = length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)/number_of_points_in_r
display(Markdown(rf"""### Properties of the chosen discretisation
* The chosen discretisation in time gives ~ {
            number_of_points_in_t/(
            laser_pulse_duration*length_of_window_for_t_normalized_to_pulse_duration/mn.ConvertPhoton(laser_wavelength,'lambdaSI','T0SI')
            )
    :.0f}
points per one laser period.
* The stepsize in the radial discretisation is ~ ${
      1e6*dr_CUPRAD
      :.2f}
~\mu {{\mathrm{{m}}}}$.
* The size of the radial computational box is ~ ${
      1e6*length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist
      :.2f}
~\mu {{\mathrm{{m}}}}$. The maximal radius of the reference Gaussian beam is ~ ${
      1e6*np.max([
            reference_Gaussian_waist*np.sqrt(1+((medium_length-reference_Gaussian_focus)/zR)**2),
            reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)
            ])
      :.2f}
~\mu {{\mathrm{{m}}}}$.$^\dagger$
* The Rayleigh length for the purely Gaussian beam is ${
      1e3*zR
      :.2f}
~{{\mathrm{{mm}}}}$ (the length of the cell is ${
      1e3*medium_length
      :.2f}
~{{\mathrm{{mm}}}}$).
* There will be ~ {
    medium_length/physical_output_distance_for_plasma_and_Efield
    :.0f} output planes.

$^\dagger$ This is given at the $z$-edges of the computational box.
"""))





In [ ]:
import XUV_refractive_index as XUV_index
import IR_refractive_index as IR_index

Horder = 17
omegaSI = mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaSI')

eta_opt = HHG.eta_opt(omegaSI,
                    XUV_index.polarisability(Horder * omegaSI,gas+'_'+XUV_dispersion_tables,),
                    IR_index.polarisability(gas,laser_wavelength))
print(eta_opt)

## Prepare the input file

Here we create the HDF5 file containing all the input parameters. First, we provide several dictionaries (for different modules) to translate the local variables usedin this jupyter notebook to the nomenclature used in the code. Second, we crreate the archive. Density modulation is treated in the second part without a dictionary.

In [ ]:
# Code to create the input hdf5-file
## First, we prepare dictionaries between hdf5-inputs and this jupyter notebook

# Code to create the input hdf5-file
## First, we prepare dictionaries between hdf5-inputs and this jupyter notebook

pre_ion_path =  MMA.global_inputs_pre_ionised_subgroup
global_input_names_to_jupyter_variables = {
    'gas_preset'                                : (np.string_(gas),                       '[-]'   ),
    'medium_pressure_in_bar'                    : (medium_pressure,                       '[bar]' ),

    pre_ion_path + '/method_geometry'           : (1,                                        '[-]'),
    pre_ion_path + '/method_units'              : (1,                                        '[-]'),
    pre_ion_path + '/initial_electrons_ratio'   : (0.,                                       '[-]'),
   
}


CUPRAD_names_to_jupyter_variables = {
    # laser parameters
    'laser_wavelength'                          : (1e2*laser_wavelength,                  '[cm]'  ),
    'laser_pulse_duration_in_1_e_Efield'        : (1e15*laser_pulse_duration,             '[fs]' ),
    'laser_focus_intensity_Gaussian'            : (reference_Gaussian_focus_intensity,    '[W/m2]'  ),
    'laser_focus_beamwaist_Gaussian'            : (reference_Gaussian_waist,              '[m]'  ),
    'laser_focus_position_Gaussian'             : (reference_Gaussian_focus,              '[m]'  ),

    # medium parameters
    'medium_physical_distance_of_propagation'   : (medium_length,                         '[m]'   ),

    # ionisation
    'ionization_model'                          : (np.string_(ionisation_model),          '[-]'  ),

    # numerics
    'numerics_number_of_points_in_r'            : (number_of_points_in_r,                 '[-]'  ),
    'numerics_number_of_points_in_t'            : (number_of_points_in_t,                 '[-]'  ),
    'numerics_operators_t_t-1'                  : (operators_t,                           '[-]'  ),
    'numerics_physical_first_stepwidth'         : (first_delta_z,                         '[mm]' ),
    'numerics_phase_threshold_for_decreasing_delta_z' : 
        (phase_threshold_for_decreasing_delta_z,                '[rad]' ),
    'numerics_length_of_window_for_r_normalized_to_beamwaist':
        (length_of_window_for_r_normalized_to_beamwaist,        '[-]'   ),
    'numerics_length_of_window_for_t_normalized_to_pulse_duration' :
        (length_of_window_for_t_normalized_to_pulse_duration,   '[-]'   ),
    'numerics_number_of_absorber_points_in_time':
        (number_of_absorber_points_in_time ,                    '[-]'   ),
    'numerics_physical_output_distance_for_plasma_and_Efield' :
        (physical_output_distance_for_plasma_and_Efield,        '[m]'   ),
    'numerics_output_distance_in_z-steps_for_fluence_and_power' :
        (output_distance_in_z_steps_for_fluence_and_power,      '[-]'   ),
    'numerics_radius_for_diagnostics'           : (radius_for_diagnostics,                '[cm]' ),
    'numerics_run_time_in_hours'                : (run_time_in_hours,                     '[s]'  )
}

global_input_names_to_jupyter_variables2 = copy.deepcopy(global_input_names_to_jupyter_variables)
global_input_names_to_jupyter_variables2[pre_ion_path + '/initial_electrons_ratio'] = (.9*eta_opt, '[-]')

## Create the hdf5-archive
from inputs_transformer import add_variables2hdf5, variables2text
outputs_path = os.path.join('/mnt','d','git','MMA-interactive')
h5filename = 'results_map1.h5'
h5filepath = os.path.join(outputs_path,h5filename)

h5filename2 = 'results_map2.h5'
h5filepath2 = os.path.join(outputs_path,h5filename2)


with h5py.File(h5filepath,'w') as f1, h5py.File(h5filepath2,'w') as f2: 

    add_variables2hdf5(f1,
                    global_input_names_to_jupyter_variables,
                    CUPRAD_names_to_jupyter_variables,
                    None,
                    None,
                    None,
                    None)

    add_variables2hdf5(f2,
                    global_input_names_to_jupyter_variables2,
                    CUPRAD_names_to_jupyter_variables,
                    None,
                    None,
                    None,
                    None)